---
title: JUNO State data pipeline
---

In [ ]:
#| export
from datetime import timedelta
import polars as pl
import pandas

from kedro.pipeline import Pipeline, node
from kedro.pipeline.modular_pipeline import pipeline

In [ ]:
#| hide
#| default_exp pipelines/juno/state
%load_ext autoreload
%autoreload 2

## Downloading data

For interpolated solar wind at JUNO's location, see [model output file](http://csem.engin.umich.edu/MSWIM2D/writable/mswim2d_interp_output_20231009173618.txt).

## Preprocessing data

```{txt}
Coordinate System:  HGI
Variables:
  Date_Time: date and time in ISO format [UT]
  hour: elapsed time since trajectory start [hr]
  r: radial coordinate in HGI [AU]
  phi: longitude coordinate in HGI [deg]
  Rho: density [amu/cm^3]
  Ux, Uy, Uz: bulk velocity components in HGI [km/s]
  Bx, By, Bz: magnetic field components in HGI [nT]
  Ti: ion temperature [K]
```

In [ ]:
# | export
def preprocess_data(
    raw_data: pandas.DataFrame,
    start: str = None,
    end: str = None,
    ts: str = None,  # time resolution
    coord: str = None,
) -> pl.DataFrame:
    """
    Preprocess the raw dataset (only minor transformations)

    - Parsing and typing data (like from string to datetime for time columns)
    - Changing storing format (like from `csv` to `parquet`)
    """
    df = (
        pl.from_pandas(raw_data)
        .lazy()
        .with_columns(
            time=pl.col("Date_Time").str.to_datetime(time_unit="ns"),
        )
        .sort("time")
        .drop(["Date_Time", "hour"])
    ).collect()
    return df

## Processing data

In [ ]:
# | export
bcols_hgi = ["bx", "by", "bz"]
bcols_rtn = ["b_r", "b_t", "b_n"]
vcols_hgi = ["ux", "uy", "uz"]
vcols_rtn = ["v_r", "v_t", "v_n"]


def hgi2rtn(df: pl.LazyFrame | pl.DataFrame):
    """Transform coordinates from HGI to RTN"""

    phi_rad = pl.col("phi_rad")
    ux = pl.col("ux")
    uy = pl.col("uy")
    uz = pl.col("uz")
    result = (
        df.with_columns(
            phi_rad=pl.col("phi").radians(),
        )
        .with_columns(
            b_r=pl.col("bx") * phi_rad.cos() + pl.col("by") * phi_rad.sin(),
            b_t=-pl.col("bx") * phi_rad.sin() + pl.col("by") * phi_rad.cos(),
            b_n=pl.col("bz"),
            v_r=ux * phi_rad.cos() + uy * phi_rad.sin(),
            v_t=-ux * phi_rad.sin() + uy * phi_rad.cos(),
            v_n=uz,
            plasma_speed=(ux**2 + uy**2 + uz**2).sqrt(),
        )
        .drop(["phi", "phi_rad"] + bcols_hgi + vcols_hgi)
    )
    return result

In [ ]:
# | export
def process_data(df: pl.DataFrame) -> pl.DataFrame:
    """
    Corresponding to primary data layer, where source data models are transformed into domain data models

    - Transforming data to RTN (Radial-Tangential-Normal) coordinate system
    - Applying naming conventions for columns
    """

    columns_name_mapping = {
        "r": "radial_distance",
        "v_r": "v_x",
        "v_t": "v_y",
        "v_n": "v_z",
        "b_r": "model_b_r",
        "b_n": "model_b_n",
        "b_t": "model_b_t",
        "Ti": "plasma_temperature",
        "rho": "plasma_density",
    }

    return df.pipe(hgi2rtn).rename(columns_name_mapping)

## Pipeline

In [ ]:
# | export
def create_pipeline(
    sat_id,
    ts: str = '1h',  # time resolution
) -> Pipeline:
    
    node_preprocess_data = node(
        preprocess_data,
        inputs=dict(
            start="params:start_date",
            end="params:end_date",
            raw_data="raw_state",
        ),
        outputs=f"inter_state_{ts}",
        name=f"preprocess_{sat_id.upper()}_state_data",
    )
    
    node_process_data = node(
        process_data,
        inputs=f"inter_state_{ts}",
        outputs=f"primary_state_{ts}",
        name=f"process_{sat_id.upper()}_state_data",
    )
    
    nodes = [
        node_preprocess_data,
        node_process_data,
    ]
    pipelines = pipeline(
        nodes,
        namespace=sat_id,
        parameters={
            "params:start_date": "params:jno_start_date",
            "params:end_date": "params:jno_end_date",
        },
    )

    return pipelines